import libraries

In [1]:
from collections import Counter
import csv
import re

from bs4 import BeautifulSoup
import janome.tokenizer
import janome.analyzer
import janome.tokenfilter
import MeCab
from tensorflow.keras.preprocessing.sequence import pad_sequences

# テキストの前処理

## テキストのクリーニング

In [3]:
!pip freeze | grep beautifulsoup4

beautifulsoup4==4.9.0


In [4]:
html = \
"""
<html>
  <body>
    これは<a href="http://example.com">Example</a>です。
  </body>
</html>
"""

### remove html tags by BeautifulSoup

In [5]:
def clean_html(html, strip=False):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(strip=strip)
    return text

In [6]:
clean_html(html)

'\n\n\n    これはExampleです。\n  \n\n'

In [7]:
clean_html(html, strip=True)

'これはExampleです。'

### use Regular Expression; 正規表現
- `r`という記号はraw stringとするためpre-fix(`\`に意味を持たせない（`print(\n)`などもそのまま`\n`と出力される）)

In [8]:
text = "今度からMkDocsでドキュメント書こう。 #Python"

In [9]:
def clean_hashtag(text):
    # hash記号に続く一文字以上の文字列
    cleaned_text = re.sub(r"#[a-zA-Z]+", "", text)
    return cleaned_text

In [10]:
clean_hashtag(text)

'今度からMkDocsでドキュメント書こう。 '

In [11]:
text = "機械学習やるなら #Python がいいよね。 #jupyter"

In [12]:
clean_hashtag(text)

'機械学習やるなら  がいいよね。 '

In [13]:
def clean_hashtag(text):
    # 文末のhash記号は続く文字も除去、文中の場合はhash記号のみ除去
    cleaned_text = re.sub(r" #[a-zA-Z]+$", "", text)
    cleaned_text = re.sub(r" #([a-zA-Z]+) ", r"\1", cleaned_text)
    return cleaned_text

In [14]:
clean_hashtag(text)

'機械学習やるならPythonがいいよね。'

In [15]:
text = "機械学習やるなら #Python がいいよね。 #jupyter #pycon #scipy"

In [16]:
clean_hashtag(text)

'機械学習やるならPythonがいいよね。jupyter#pycon'

In [17]:
def clean_hashtag(text):
    # 文末のhash記号は続く文字も除去、文中の場合はhash記号のみ除去、hash記号が連続する場合はそれらをまとめて除去
    cleaned_text = re.sub(r"( #[a-zA-Z]+)+$", "", text)
    cleaned_text = re.sub(r" #([a-zA-Z]+) ", r"\1", cleaned_text)
    return cleaned_text

In [18]:
clean_hashtag(text)

'機械学習やるならPythonがいいよね。'

## 単語の分割

In [2]:
text = "彼女と国立新美術館へ行った。"

### in case use janome

In [20]:
# from janome.tokenizer import Tokenizer をしていれば Tokenizer() のみでおk、今回はモジュールを把握するためフルで記述している
t = janome.tokenizer.Tokenizer()
for token in t.tokenize(text):
    print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,格助詞,一般,*,*,*,と,ト,ト
国立	名詞,一般,*,*,*,*,国立,コクリツ,コクリツ
新	接頭詞,名詞接続,*,*,*,*,新,シン,シン
美術館	名詞,一般,*,*,*,*,美術館,ビジュツカン,ビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


In [21]:
t = janome.tokenizer.Tokenizer(wakati=True)
t.tokenize(text)

['彼女', 'と', '国立', '新', '美術館', 'へ', '行っ', 'た', '。']

特定の瀕死の単語のみ抽出（フィルタリング）

In [22]:
token_filters = [
    janome.tokenfilter.POSKeepFilter("名詞")
]
a = janome.analyzer.Analyzer(token_filters=token_filters)
for token in a.analyze(text):
    print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
国立	名詞,一般,*,*,*,*,国立,コクリツ,コクリツ
美術館	名詞,一般,*,*,*,*,美術館,ビジュツカン,ビジュツカン


<u>**辞書への単語の追加（MeCabでも同様の手法で可能）**</u>  
```
表層形、左文脈ID、右文脈ID、コスト、品詞、品詞再分類1、品詞再分類2、品詞再分類3、活用型、活用形、原型、読み、発音
```

In [23]:
with open("data/chapter04_userdic.csv", "w", encoding="utf-8") as f:
    userdic_word = [
        ["国立新美術館", "1288", "1288", "100", "名詞", "固有名詞", 
         "一般", "*", "*", "*", "国立新美術館", "コクリツシンビジュツカン","コクリツシンビジュツカン"
        ]
    ]
    writer = csv.writer(f)
    writer.writerows(userdic_word)

In [24]:
t = janome.tokenizer.Tokenizer(udic="data/chapter04_userdic.csv", udic_enc="utf8")
for token in t.tokenize(text):
    print(token)

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,並立助詞,*,*,*,*,と,ト,ト
国立新美術館	名詞,固有名詞,一般,*,*,*,国立新美術館,コクリツシンビジュツカン,コクリツシンビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。


### in case use MaCab

In [25]:
t_mecab = MeCab.Tagger("")
print(t_mecab.parse(text))

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,格助詞,一般,*,*,*,と,ト,ト
国立	名詞,一般,*,*,*,*,国立,コクリツ,コクリツ
新	接頭詞,名詞接続,*,*,*,*,新,シン,シン
美術館	名詞,一般,*,*,*,*,美術館,ビジュツカン,ビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



`mecab-ipadic-NEologd`を使用

In [3]:
path_neologd = !echo `mecab-config --dicdir`"/mecab-ipadic-neologd"
path_neologd

['/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd']

In [27]:
t_mecab = MeCab.Tagger(
    "-d {}".format(path_neologd[0])
)
print(t_mecab.parse(text))

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,並立助詞,*,*,*,*,と,ト,ト
国立新美術館	名詞,固有名詞,地域,一般,*,*,国立新美術館,コクリツシンビジュツカン,コクリツシンビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [22]:
t_mecab = MeCab.Tagger(
    "-d {}".format(path_neologd[0])
)
print(t_mecab.parse(text))

彼女	名詞,代名詞,一般,*,*,*,彼女,カノジョ,カノジョ
と	助詞,並立助詞,*,*,*,*,と,ト,ト
国立新美術館	名詞,固有名詞,地域,一般,*,*,国立新美術館,コクリツシンビジュツカン,コクリツシンビジュツカン
へ	助詞,格助詞,一般,*,*,*,へ,ヘ,エ
行っ	動詞,自立,*,*,五段・カ行促音便,連用タ接続,行く,イッ,イッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [21]:
t_mecab = MeCab.Tagger(
    "-Owakati && -d {}".format(path_neologd[0])
)
print(list(t_mecab.parse(text).strip().split(" ")))

['彼女', 'と', '国立新美術館', 'へ', '行っ', 'た', '。']


In [28]:
#t_mecab.parse("") # あってもなくても一緒
m = t_mecab.parseToNode(text)

while m:
    if m.feature.split(',')[0] == '名詞':
        print(m.surface)
    m = m.next

彼女
国立新美術館


## 単語の正規化

### 文字種の統一  
- 小文字化 `lower()`
- 大文字化 `upper()`
- 先頭のみ大文字 `title()`

In [29]:
text = "President Obama is speaking at the White House."
print("lower: ", text.lower())
print("upper: ", text.upper())
print("title: ", text.title())

lower:  president obama is speaking at the white house.
upper:  PRESIDENT OBAMA IS SPEAKING AT THE WHITE HOUSE.
title:  President Obama Is Speaking At The White House.


### 数字の置き換え
数字を表す正規表現（厳密には、ひとつ以上の連続した数値）は`\d+`

In [30]:
def normalize_number(text, same_length=False):
    replaced_text = re.sub(r"\d" if same_length else r"\d+", "0", text)
    return replaced_text

text = "2万0689・24ドル"
print("replace into one-zero: ", normalize_number(text))
print("          same length: ", normalize_number(text, same_length=True))

replace into one-zero:  0万0・0ドル
          same length:  0万0000・00ドル


## ストップワードの除去
出現頻度が高い割に自然言語処理には役に立たない言葉：「は」「が」「です」「ます」etc.

### 辞書による方式
[Slothlib](http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt)を使用 -> `Japanese.txt`

In [ ]:
!wget http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt -P data/

In [31]:
with open("data/Japanese.txt", "r", encoding="utf-8") as f:
    stopwords = [w.strip() for w in f] # remove \n
    stopwords = set(stopwords) # set array is faster than list one

In [32]:
def remove_stopwords(words, stopwords):
    words = [w for w in words if w not in stopwords]
    return words

In [33]:
t = janome.tokenizer.Tokenizer(wakati=True)
text = "りんごをいくつか買う。"
words = t.tokenize(text)
print("before remove stopwords: ", words)
words = remove_stopwords(words, stopwords)
print(" after remove stopwords: ", words)

before remove stopwords:  ['りんご', 'を', 'いくつ', 'か', '買う', '。']
 after remove stopwords:  ['りんご', 'を', 'か', '買う', '。']


### 出現頻度による方式
頻度をカウントする対象のコーパスとして[ja.text8](https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip)を使用  
（なお、教本でのURL誤植だった模様(正：`http://s3-ap-northeast-1.,,,`, 誤：`http://s3-apnortheast-1.,,,`)）

In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip -P data/
!find ./data -name \*.zip -type f | sed 's#\(.*\)\.zip#mv & \1.gz#' | sh
!gunzip -d data/ja.text8.gz

In [34]:
with open("data/ja.text8", "r", encoding="utf-8") as f:
    text = f.read()
    words = text.split()

、、、いやめちゃ重。メモリの使用量チェック。

In [35]:
import sys
import pandas as pd

mem_cols = ['Variable Name', 'Memory']
memory_df = pd.DataFrame(columns=mem_cols)

for var_name in dir():
    if not var_name.startswith("_"):
        memory_df = memory_df.append(pd.DataFrame([[var_name, sys.getsizeof(eval(var_name))]], columns=mem_cols))

memory_df = memory_df.sort_values(by='Memory', ascending=False).reset_index(drop=True)
display(memory_df)

,Variable Name,Memory
0,text,186031248
1,words,139361088
2,stopwords,32992
3,memory_df,2088
4,BeautifulSoup,2000
5,Counter,1056
6,NamespaceMagics,888
7,Out,648
8,In,432
9,html,246


In [36]:
del text # release memory

In [37]:
Counter(["cat", "dog", "cat"])

Counter({'cat': 2, 'dog': 1})

In [38]:
fdist = Counter(words)
fdist.most_common(n=10) # top n words

[('の', 828585),
 ('、', 785716),
 ('。', 532921),
 ('に', 527014),
 ('は', 488009),
 ('を', 423115),
 ('た', 421908),
 ('が', 353221),
 ('で', 350821),
 ('て', 259995)]

In [39]:
del words # release memory !! CAUTION !! fdist will be used below; not neccessary delete fdist

抽出された高頻度な単語を`stopwords`として使用可能

In [40]:
t = janome.tokenizer.Tokenizer(wakati=True)
text = "りんごをいくつか買う。"
words = t.tokenize(text)
print("before remove stopwords: ", words)
words = remove_stopwords(words, np.array(fdist.most_common(n=10))[:,0])
print(" after remove stopwords: ", words)

before remove stopwords:  ['りんご', 'を', 'いくつ', 'か', '買う', '。']
 after remove stopwords:  ['りんご', 'いくつ', 'か', '買う']


## 単語のID化
IDを割り当てた辞書を**ボキャブラリ**と呼んだりなんかしちゃう

In [41]:
UNK = "<UNK>" # UNKnown;実際に処理する文章に出現する単語がボキャブラリvocabに含まれていないこともあるため、登録しておく単語ID
PAD = "<PAD>" # PADding
vocab = {PAD:0, UNK:1}
for word, _ in fdist.most_common():
    vocab[word] = len(vocab)

In [42]:
words = ["私", "は", "元気"]
word_ids = [vocab.get(w, vocab[UNK]) for w in words]
print(word_ids)

[1151, 6, 7901]


## パディング
入力にデータを加えてデータ系列長を合わせること  
e.g.)  
- [I, love, you] ... 系列長3
- [love] ... 系列長1
- [love, \<PAD>, \<PAD>] ... 系列長3(パディング後)

In [43]:
sequences = [
    [1,2], [3,4,5], [6,7,8,9]
]
print("dtype; ", pad_sequences(sequences).dtype)
print("pre-padding; \n", pad_sequences(sequences))
print("post-padding; \n", pad_sequences(sequences, padding="post"))
print("post-padding (specified length = 3, truncuting from pre); \n", 
      pad_sequences(sequences, padding="post", maxlen=3))
print("post-padding (specified length = 3, truncuting from post); \n", 
      pad_sequences(sequences, padding="post", maxlen=3, truncating="post"))
print("pre-padding (specified length = 3, truncuting from pre, padding by '10'); \n", 
      pad_sequences(sequences, maxlen=3, truncating="pre", value=10))
print("post-padding (specified length = 3, truncuting from post, padding by '10'); \n", 
      pad_sequences(sequences, padding="post", maxlen=3, truncating="post", value=10))

dtype;  int32
pre-padding; 
 [[0 0 1 2]
 [0 3 4 5]
 [6 7 8 9]]
post-padding; 
 [[1 2 0 0]
 [3 4 5 0]
 [6 7 8 9]]
post-padding (specified length = 3, truncuting from pre); 
 [[1 2 0]
 [3 4 5]
 [7 8 9]]
post-padding (specified length = 3, truncuting from post); 
 [[1 2 0]
 [3 4 5]
 [6 7 8]]
pre-padding (specified length = 3, truncuting from pre, padding by '10'); 
 [[10  1  2]
 [ 3  4  5]
 [ 7  8  9]]
post-padding (specified length = 3, truncuting from post, padding by '10'); 
 [[ 1  2 10]
 [ 3  4  5]
 [ 6  7  8]]


# 前処理の実践
データセットの学習と評価（モデルの学習は後の章）

<span style="color:red"><b>
    ここはscriptを作成した<br>
    なお、`Amazon Customer Reviews Dataset - amazon_reviews_multilingual1_JP_v1_00.tsv`は[ここ](http://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual1_JP_v1_00.tsv.gz)からDLした<br>
    [サポートサイトのgoogle colab](https://gist.github.com/Hironsan/1f1cc629613cbd7de042a7ce269b91d6)にterminalでDLするためのコードが載っていた
    </b></span>

In [44]:
!wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz -P data/
!gunzip -d data/amazon_reviews_multilingual_JP_v1_00.tsv.gz

--2020-05-08 14:17:01--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.242.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.242.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94688992 (90M) [application/x-gzip]
Saving to: ‘data/amazon_reviews_multilingual_JP_v1_00.tsv.gz’

amazon_reviews_mult 100%[===================>]  90.30M   961KB/s    in 2m 16s  

2020-05-08 14:19:18 (681 KB/s) - ‘data/amazon_reviews_multilingual_JP_v1_00.tsv.gz’ saved [94688992/94688992]



In [45]:
import pandas as pd
try:
    pd.read_csv("data/amazon_reviews_multilingual_JP_v1_00.tsv", delimiter="\t").head()
except FileNotFoundError: 
    # after file move to src/scripts/chapter04/data
    pd.read_csv("src/scripts/chapter04/data/amazon_reviews_multilingual_JP_v1_00.tsv", delimiter="\t").head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,JP,65317,R33RSUD4ZTRKT7,B000001GBJ,957145596,SONGS FROM A SECRET GARDE,Music,1,1,15,N,Y,残念ながら…,残念ながら…趣味ではありませんでした。ケルト音楽の範疇にも幅があるのですね…,2012-12-05
1,JP,65317,R2U1VB8GPZBBEH,B000YPWBQ2,904244932,鏡の中の鏡‾ペルト作品集(SACD)(Arvo Part:Spiegel im Spiegel),Music,1,4,20,N,Y,残念ながら…,残念ながら…趣味ではありませんでした。正直退屈…眠気も起きない…,2012-12-05
2,JP,65696,R1IBRCJPPGWVJW,B0002E5O9G,108978277,Les Miserables 10th Anniversary Concert,Music,5,2,3,N,Y,ドリームキャスト,素晴らしいパフォーマンス。ミュージカル映画版の物足りない歌唱とは違います。,2013-03-02
3,JP,67162,RL02CW5XLYONU,B00004SRJ5,606528497,It Takes a Nation of Millions to Hold Us Back,Music,5,6,9,N,Y,やっぱりマスト,専門的な事を言わずにお勧めレコメを書きたいのですが、文才が無いので無理でした。ヒップホップが...,2013-08-11
4,JP,67701,R2LA2SS3HU3A3L,B0093H8H8I,509738390,Intel CPU Core I3-3225 3.3GHz 3MBキャッシュ LGA1155...,PC,4,2,4,N,Y,コスパ的には十分,今までの環境（Core2 Duo E4600)に比べれば十分に快適になりました。<br />...,2013-02-10
